# User Authentication using OAuth2 and APIM fronting Azure OpenAI

In [8]:
tenant_id = "YOUR_ENTRA_TENANT" 
client_id = "YOUR_CLIENT_ID" 
apim_endpoint = "https://YOUR_APIM_ENDPOINT.azure-api.net"
apim_key = "YOUR_APIM_SUBSCRIPTION" # an APIM subscription key
openai_deployment_name = "gpt-4o"
openai_api_version = "2024-06-01"


In [ ]:
%pip install msal
%pip install openai
%pip install requests
%pip install logging
%pip install json

##  Create a device flow to get the access token

In [ ]:
import json
import logging

import requests
import msal

app = msal.PublicClientApplication(
    client_id, authority="https://login.microsoftonline.com/" + tenant_id)

flow = app.initiate_device_flow(scopes=["User.Read"])
if "user_code" not in flow:
    raise ValueError(
        "Fail to create device flow. Err: %s" % json.dumps(flow, indent=4))

print(flow["message"])

## Acquire the token and query the graph API

In [ ]:
result = app.acquire_token_by_device_flow(flow)
print(result) # debug
if "access_token" in result:
    access_token = result['access_token']
    # Calling graph using the access token
    graph_data = requests.get(  # Use token to call downstream service
        "https://graph.microsoft.com/v1.0/me",
        headers={'Authorization': 'Bearer ' + access_token},).json()
    print("Graph API call result: %s" % json.dumps(graph_data, indent=2))
    # print(access_token) # Use a tool like https://jwt.io/ to decode the access token and see its contents
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))  # You may need this when reporting a bug

## Test the API using a direct HTTP call

In [ ]:
url = apim_endpoint + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
response = requests.post(url, headers = {'api-key':apim_key, 'Authorization': 'Bearer ' + access_token}, json = messages)
print("status code: ", response.status_code)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)

## Test the API using the Azure OpenAI Python SDK

In [ ]:
from openai import AzureOpenAI

messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apim_endpoint,
    api_key=apim_key,
    api_version=openai_api_version        
)
response = client.chat.completions.create(model=openai_deployment_name, messages=messages, extra_headers={"Authorization": "Bearer " + access_token})
print(response.choices[0].message.content)